In [5]:
from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

import os

import glob
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import urllib.request
import json
import codecs
import facebook
from google2pandas import *


InteractiveShell.ast_node_interactivity = "all"

In [6]:
## Baseline Variablen

shares_mean = 66.8
pageviews_mean = 2045.5
comments_mean = 6.6
readtime_mean = 186.6

## Setup Weeks and Stats Range

In [7]:
id = 5
week = 'KW ' + str(id)
start = '2017-01-30'
end_week = '2017-02-05'
end_stats = '2017-02-12'

## Load and prepare google stats

In [24]:
conn = GoogleAnalyticsQuery(token_file_name='analytics.dat', secrets='client_secrets_v3.json')


Failed to start a local webserver listening on either port 8080
or port 8090. Please check your firewall settings and locally
running programs that may be blocking or using those ports.

Falling back to --noauth_local_webserver and continuing with
authorization.


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=944262119650-5q06vgihu1eedniu14nl6rhu53srjart.apps.googleusercontent.com&response_type=code&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob

Enter verification code: 4/cC-QjZkWgrMcj6ASr6t5pItCOgvpNWUeZpTv_rOFWls
Authentication successful.


In [25]:
query_pageviews = {\
    'ids' : '48980166',
    'metrics' : ['pageviews', 'avgTimeOnPage'],
    'dimensions' : 'pagePath',
    'start_date' : start,
    'end_date' : end_stats,
    'sort' : '-pageviews',
    'max_results' : 5000
    }

In [26]:
query_user = {\
    'ids' : '48980166',
    'metrics' : 'users',
    'start_date' : start,
    'end_date' : end_week,
    }

In [27]:
users, metadata = conn.execute_query(**query_user)

In [28]:
users

,users
0,85302


In [672]:
stats_weekly, metadata = conn.execute_query(**query_pageviews)

In [673]:
stats_weekly.sort_values('pageviews', ascending=False).head()

,pagePath,pageviews,avgTimeOnPage
0,/,139815,104.17672591761331
1,/de/2017_5/basel/741541,21990,161.98881620595063
2,/de/2017_5/basel/741458,9772,304.6311143802869
3,/de/2017_6/basel/,8004,63.56938184159691
4,/de/blogs/bildstoff/742199,7694,316.13870967741934


In [674]:
stats_weekly['url'] = stats_weekly['pagePath'].str.split('/')

for i in range(0, len(stats_weekly['url'])):   
    try:
        stats_weekly.loc[i, 'article_id'] = stats_weekly['url'][i][4]
    except:
        stats_weekly.loc[i, 'article_id'] = ''
        
stats_weekly['article_id'] = pd.to_numeric(stats_weekly['article_id'], errors='coerce')
stats_weekly['pageviews'] = pd.to_numeric(stats_weekly['pageviews'], errors='coerce')
stats_weekly['avgTimeOnPage'] = pd.to_numeric(stats_weekly['avgTimeOnPage'], errors='coerce')


stats_weekly = stats_weekly.dropna()

In [675]:
stats_readtime = stats_weekly.groupby(by=['article_id']).avgTimeOnPage.mean()
stats_readtime = stats_readtime.to_frame(name = 'readtime')
stats_readtime = stats_readtime.reset_index()

stats_readtime.head()

,article_id,readtime
0,1.0,60.500000
1,3915.0,148.210526
2,3917.0,56.284404
3,3918.0,134.602151
4,3919.0,49.826087


In [676]:
stats_pageviews = stats_weekly.groupby(by=['article_id']).pageviews.sum()
stats_pageviews = stats_pageviews.to_frame(name = 'pageviews')
stats_pageviews = stats_pageviews.reset_index()

stats_pageviews.head()

,article_id,pageviews
0,1.0,34
1,3915.0,30
2,3917.0,173
3,3918.0,197
4,3919.0,29


In [677]:
stats_merged = stats_pageviews.merge(stats_readtime, left_on='article_id', right_on='article_id')

In [678]:
stats = stats_merged[['article_id', 'pageviews', 'readtime']]

In [679]:
stats.head()

,article_id,pageviews,readtime
0,1.0,34,60.500000
1,3915.0,30,148.210526
2,3917.0,173,56.284404
3,3918.0,197,134.602151
4,3919.0,29,49.826087


## Setup metaInfo

In [680]:
metaInfo_current = pd.DataFrame(columns=['id', 'week', 'nutzer', 'published_articles', 
                                 'pageviews_news', 'pageviews_newswire', 'pageviews_total', 'facebook_interaktionen'])

metaInfo_new = ['', '', '', '', '', '', '', '']
metaInfo_current.loc[len(metaInfo_current)] = metaInfo_new

In [683]:
metaInfo_current['week'] = week
metaInfo_current['id'] = id

metaInfo_current['nutzer'] = users['users']

metaInfo_current

,id,week,nutzer,published_articles,pageviews_news,pageviews_newswire,pageviews_total,facebook_interaktionen
0,5,KW 5,85302,,,,,


## Get meta data of articles

In [684]:
def parse_articles(id):
    meta = pd.DataFrame(columns=['type', 'article_id'])
    url = 'http://www.tageswoche.ch/content-api/articles/'
    article_name = url + str(id)
    #print(id)
    try:
        df = pd.read_json(article_name, lines=True)
        meta['type'] = df['type']
        meta['published'] = df['published']
        meta['title'] = df['title']
        meta['url'] = df['url']
        meta['comments'] = df['comments_count']
    except:
        pass
    meta['article_id'] = id
    return meta

filepath = 'weekly_report/tawo_stats_meta_' + week + '.csv'

if os.path.exists(filepath):
    df = pd.read_csv(filepath, usecols=[1, 2, 3, 4, 5, 6])
else:
    df = pd.concat([parse_articles(id) for id in stats['article_id']])

df.head()

,article_id,comments,published,title,type,url
0,3915.0,0.0,2011-10-07T12:43:09+0200,AGB,static_page,http://www.tageswoche.ch/de/pages/about/3915/a...
0,3917.0,0.0,2011-10-07T12:43:59+0200,Impressum,static_page,http://www.tageswoche.ch/de/pages/about/3917/i...
0,3918.0,0.0,2011-10-07T12:44:54+0200,Kontakt,static_page,http://www.tageswoche.ch/de/pages/about/3918/k...
0,3919.0,0.0,2011-10-07T12:44:43+0200,Community-Leitfaden,static_page,http://www.tageswoche.ch/de/pages/about/3919/c...
0,3923.0,0.0,2011-10-07T12:46:09+0200,Über uns,static_page,http://www.tageswoche.ch/de/pages/about/3923/u...


In [685]:
df['published'] = df['published'].str.extract('(\d\d\d\d-\d\d-\d\d)')

df = df[df.published != '0001-11-30']

df['published'] = pd.to_datetime(df['published'], format='%Y-%m-%d')

df.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


,article_id,comments,published,title,type,url
0,3915.0,0.0,2011-10-07,AGB,static_page,http://www.tageswoche.ch/de/pages/about/3915/a...
0,3917.0,0.0,2011-10-07,Impressum,static_page,http://www.tageswoche.ch/de/pages/about/3917/i...
0,3918.0,0.0,2011-10-07,Kontakt,static_page,http://www.tageswoche.ch/de/pages/about/3918/k...
0,3919.0,0.0,2011-10-07,Community-Leitfaden,static_page,http://www.tageswoche.ch/de/pages/about/3919/c...
0,3923.0,0.0,2011-10-07,Über uns,static_page,http://www.tageswoche.ch/de/pages/about/3923/u...


In [686]:
df.to_csv(filepath)

In [687]:
# merge google stats and article meta data

stats_metainfo = df.merge(stats, left_on='article_id', right_on='article_id')

#df = df.drop_duplicates()

stats_metainfo.head()

,article_id,comments,published,title,type,url,pageviews,readtime
0,3915.0,0.0,2011-10-07,AGB,static_page,http://www.tageswoche.ch/de/pages/about/3915/a...,30,148.210526
1,3917.0,0.0,2011-10-07,Impressum,static_page,http://www.tageswoche.ch/de/pages/about/3917/i...,173,56.284404
2,3918.0,0.0,2011-10-07,Kontakt,static_page,http://www.tageswoche.ch/de/pages/about/3918/k...,197,134.602151
3,3919.0,0.0,2011-10-07,Community-Leitfaden,static_page,http://www.tageswoche.ch/de/pages/about/3919/c...,29,49.826087
4,3923.0,0.0,2011-10-07,Über uns,static_page,http://www.tageswoche.ch/de/pages/about/3923/u...,103,42.695122


## Prepare subsets for news and newswire

In [688]:
stats_metainfo['type'] = stats_metainfo['type'].astype('category')

stats_metainfo_news = stats_metainfo[(stats_metainfo.type == 'news')]

stats_metainfo_news.head()

,article_id,comments,published,title,type,url,pageviews,readtime
6,80382.0,0.0,2011-10-27,Die TagesWoche: Mehr als eine Zeitung,news,http://www.tageswoche.ch/de/2011_43/dialog/803...,7,185.0
7,80423.0,0.0,2011-10-27,Schweizerhalle: Die Lehren aus dem Unfall,news,http://www.tageswoche.ch/de/2011_43/basel/8042...,10,0.0
9,104505.0,7.0,2011-11-04,"Adieu, Anarchie! Basels Feierszene wird flügge",news,http://www.tageswoche.ch/de/2011_44/kultur/104...,11,221.0
10,104531.0,1.0,2011-11-04,Viren und Kokain auf den meisten Geldscheinen,news,http://www.tageswoche.ch/de/2011_44/schweiz/10...,4,0.0
11,104558.0,0.0,2011-11-02,Zürcher Daros-Museum schliesst für immer,news,http://www.tageswoche.ch/de/2011_43/kultur/104...,4,0.0


In [689]:
data_subset_news = stats_metainfo_news.set_index(['published'])

data_subset_news_week = data_subset_news.loc[start:end_week]

In [690]:
data_subset_news_week = data_subset_news_week.reset_index()

In [691]:
# Save metainfos

metaInfo_current['pageviews_news'] = data_subset_news_week['pageviews'].sum()

In [692]:
# Save metainfos

metaInfo_current['published_articles'] = data_subset_news_week['article_id'].nunique()

## Get Facebook Shares

In [693]:
token = 'EAAaZBFx0Bj1wBAO63XZCLOOheplxnz6oABZBe9CSODEJDvDgLDY8hsMhJR81LZBMnHHDWfZBLSf33FpxUIAn6SDXAULrU0DOUpBiBhuvzFmZAHoLP9aIzqJVF9WdpCckSX5ZAHdXvDPZAZCp3nUxtDO7bwfza8TMNNNgZD'
graph = facebook.GraphAPI(access_token=token, version='2.8')

#app_id = '1897856340430684' # Obtained from https://developers.facebook.com/
#app_secret = 'afd88726fc4b0c589e30f5cd1293b92e' # Obtained from https://developers.facebook.com/

# Extend the expiration time of a valid OAuth access token.
#extended_token = graph.extend_access_token(app_id, app_secret)
#extended_token #verify that it expires in 60 days

In [694]:
def get_facebook_shares(url):
    shares = pd.DataFrame(columns=['url', 'share_count'])
    shares_new = ['', '']  
    shares.loc[len(shares)] = shares_new
    try:
        post = graph.get_object(id=url)
        shares['share_count'] = post['share']['share_count']
        shares['url'] = url
    except:
        pass
        
    return shares

share_stats = pd.concat([get_facebook_shares(url) for url in data_subset_news_week['url']])

share_stats = share_stats.reset_index()
share_stats = share_stats[[1,2]]

share_stats.head()

,url,share_count
0,http://www.tageswoche.ch/de/2017_4/leben/74068...,4
1,http://www.tageswoche.ch/de/2017_4/schweiz/740...,49
2,http://www.tageswoche.ch/de/2017_4/basel/74094...,10
3,http://www.tageswoche.ch/de/2017_4/leben/74095...,38
4,http://www.tageswoche.ch/de/2017_4/basel/74107...,74


In [695]:
metaInfo_current['facebook_interaktionen'] = share_stats['share_count'].sum()

In [696]:
df_newswire = stats_metainfo[(stats_metainfo.type == 'newswire')]

data_subset_newswire = df_newswire.set_index(['published'])

data_subset_newswire_week = data_subset_newswire.loc[start:end_week]

In [697]:
# Save metainfos

metaInfo_current['pageviews_newswire'] = data_subset_newswire_week['pageviews'].sum()

In [698]:
# Save metainfos

metaInfo_current['pageviews_total'] = metaInfo_current['pageviews_news'] + metaInfo_current['pageviews_newswire']

In [699]:
# Show metainfos

if os.path.exists('weekly_report/metainfo.csv'):
    metaInfo_old = pd.read_csv('weekly_report/metainfo.csv', usecols=[1,2,3,4,5,6,7,8])
    metaInfo = metaInfo_old.append(metaInfo_current, ignore_index=True)
else:
    metaInfo = metaInfo_current

metaInfo.to_csv('weekly_report/metainfo.csv')

metaInfo

,id,week,nutzer,published_articles,pageviews_news,pageviews_newswire,pageviews_total,facebook_interaktionen
0,1,KW 1,86754,47,84872,17986,102858,1403
1,2,KW 2,78513,53,86146,16992,103138,2556
2,3,KW 3,93401,53,91479,17245,108724,3034
3,4,KW 4,90793,57,83713,26505,110218,2280
4,5,KW 5,85302,53,90931,15247,106178,1325


## Erfolgsindex

In [700]:
data_subset_erfolgsindex = data_subset_news_week.merge(share_stats, left_on='url', right_on='url')
data_subset_erfolgsindex.head()

,published,article_id,comments,title,type,url,pageviews,readtime,share_count
0,2017-02-02,740680.0,3.0,Im virtuellen Hotel Vue des Alpes ist Perfekti...,news,http://www.tageswoche.ch/de/2017_4/leben/74068...,572,116.559759,4
1,2017-01-30,740733.0,29.0,Rechnen mit Ueli,news,http://www.tageswoche.ch/de/2017_4/schweiz/740...,2325,93.705642,49
2,2017-01-30,740944.0,9.0,System Vertrauen: Die Organisation bei Chemie-...,news,http://www.tageswoche.ch/de/2017_4/basel/74094...,1394,141.468558,10
3,2017-01-31,740955.0,6.0,Wie Mediziner der Uni Basel sich in der virtue...,news,http://www.tageswoche.ch/de/2017_4/leben/74095...,1173,168.536336,38
4,2017-01-30,741074.0,0.0,"Reiseführer durchs versteckte Basel, Teil 2",news,http://www.tageswoche.ch/de/2017_4/basel/74107...,1514,118.158977,74


In [701]:
data_subset_erfolgsindex['kennzahl_shares'] = (data_subset_erfolgsindex['share_count'] / shares_mean)*0.3
data_subset_erfolgsindex['kennzahl_comments'] = (data_subset_erfolgsindex['comments'] / comments_mean)*0.15
data_subset_erfolgsindex['kennzahl_readtime'] = (data_subset_erfolgsindex['readtime'] / readtime_mean)*0.15
data_subset_erfolgsindex['kennzahl_pageviews'] = (data_subset_erfolgsindex['pageviews'] / pageviews_mean)*0.4


data_subset_erfolgsindex.head()

,published,article_id,comments,title,type,url,pageviews,readtime,share_count,kennzahl_shares,kennzahl_comments,kennzahl_readtime,kennzahl_pageviews
0,2017-02-02,740680.0,3.0,Im virtuellen Hotel Vue des Alpes ist Perfekti...,news,http://www.tageswoche.ch/de/2017_4/leben/74068...,572,116.559759,4,0.017964,0.068182,0.093698,0.111855
1,2017-01-30,740733.0,29.0,Rechnen mit Ueli,news,http://www.tageswoche.ch/de/2017_4/schweiz/740...,2325,93.705642,49,0.220060,0.659091,0.075326,0.454657
2,2017-01-30,740944.0,9.0,System Vertrauen: Die Organisation bei Chemie-...,news,http://www.tageswoche.ch/de/2017_4/basel/74094...,1394,141.468558,10,0.044910,0.204545,0.113721,0.272598
3,2017-01-31,740955.0,6.0,Wie Mediziner der Uni Basel sich in der virtue...,news,http://www.tageswoche.ch/de/2017_4/leben/74095...,1173,168.536336,38,0.170659,0.136364,0.135479,0.229382
4,2017-01-30,741074.0,0.0,"Reiseführer durchs versteckte Basel, Teil 2",news,http://www.tageswoche.ch/de/2017_4/basel/74107...,1514,118.158977,74,0.332335,0.000000,0.094983,0.296065


In [702]:
data_subset_erfolgsindex['erfolgsindex'] = data_subset_erfolgsindex['kennzahl_shares'] + data_subset_erfolgsindex['kennzahl_pageviews'] + data_subset_erfolgsindex['kennzahl_comments'] + data_subset_erfolgsindex['kennzahl_readtime'] 

In [703]:
#data_subset_erfolgsindex = data_subset_erfolgsindex[['title', 'url', 'erfolgsindex']]

data_subset_erfolgsindex

,published,article_id,comments,title,type,url,pageviews,readtime,share_count,kennzahl_shares,kennzahl_comments,kennzahl_readtime,kennzahl_pageviews,erfolgsindex
0,2017-02-02,740680.0,3.0,Im virtuellen Hotel Vue des Alpes ist Perfekti...,news,http://www.tageswoche.ch/de/2017_4/leben/74068...,572,116.559759,4,0.017964,0.068182,0.093698,0.111855,0.291699
1,2017-01-30,740733.0,29.0,Rechnen mit Ueli,news,http://www.tageswoche.ch/de/2017_4/schweiz/740...,2325,93.705642,49,0.220060,0.659091,0.075326,0.454657,1.409133
2,2017-01-30,740944.0,9.0,System Vertrauen: Die Organisation bei Chemie-...,news,http://www.tageswoche.ch/de/2017_4/basel/74094...,1394,141.468558,10,0.044910,0.204545,0.113721,0.272598,0.635775
3,2017-01-31,740955.0,6.0,Wie Mediziner der Uni Basel sich in der virtue...,news,http://www.tageswoche.ch/de/2017_4/leben/74095...,1173,168.536336,38,0.170659,0.136364,0.135479,0.229382,0.671883
4,2017-01-30,741074.0,0.0,"Reiseführer durchs versteckte Basel, Teil 2",news,http://www.tageswoche.ch/de/2017_4/basel/74107...,1514,118.158977,74,0.332335,0.000000,0.094983,0.296065,0.723383
5,2017-01-30,741175.0,3.0,«Aaah!» und «Ooh!» im Haus der elektronischen ...,news,http://www.tageswoche.ch/de/2017_4/kultur/7411...,809,140.090824,37,0.166168,0.068182,0.112613,0.158201,0.505164
6,2017-01-30,741178.0,3.0,Basel hat einen neuen Trommelkönig – und ein j...,news,http://www.tageswoche.ch/de/2017_4/basel/74117...,957,186.581111,25,0.112275,0.068182,0.149985,0.187143,0.517585
7,2017-01-30,741240.0,0.0,Alles überwacht? Alles im Griff? Alles Roger,news,http://www.tageswoche.ch/de/2017_5/basel/74124...,615,134.257143,0,0.000000,0.000000,0.107924,0.120264,0.228188
8,2017-01-30,741251.0,2.0,Ein Bild von einer Stadt,news,http://www.tageswoche.ch/de/2017_5/basel/74125...,1506,57.216592,24,0.107784,0.045455,0.045994,0.294500,0.493733
9,2017-01-30,741255.0,3.0,Das Ende des Tennis-Gottes,news,http://www.tageswoche.ch/de/2017_5/sport/74125...,2539,192.744012,18,0.080838,0.068182,0.154939,0.496505,0.800464


In [704]:
top10 = data_subset_erfolgsindex.sort_values('erfolgsindex', ascending=False).head(15)

top10

,published,article_id,comments,title,type,url,pageviews,readtime,share_count,kennzahl_shares,kennzahl_comments,kennzahl_readtime,kennzahl_pageviews,erfolgsindex
39,2017-02-02,741541.0,9.0,Entlassung bei der «Basler Zeitung»,news,http://www.tageswoche.ch/de/2017_5/basel/74154...,23379,115.598436,78,0.350299,0.204545,0.092925,4.571792,5.219561
29,2017-02-02,741458.0,27.0,«Basler Zeitung» schreibt Medienmitteilung für...,news,http://www.tageswoche.ch/de/2017_5/basel/74145...,10870,256.366830,400,1.796407,0.613636,0.206083,2.125642,4.741768
1,2017-01-30,740733.0,29.0,Rechnen mit Ueli,news,http://www.tageswoche.ch/de/2017_4/schweiz/740...,2325,93.705642,49,0.220060,0.659091,0.075326,0.454657,1.409133
45,2017-02-05,741626.0,43.0,Orwells «1984» feiert Comeback in Trumps 2017,news,http://www.tageswoche.ch/de/2017_5/internation...,790,160.644954,19,0.085329,0.977273,0.129136,0.154485,1.346223
14,2017-01-31,741282.0,26.0,Feinstaub-Grenzwerte in Basel überschritten,news,http://www.tageswoche.ch/de/2017_5/basel/74128...,1920,138.892541,14,0.062874,0.590909,0.111650,0.375458,1.140892
22,2017-02-04,741366.0,15.0,Ein Spruch unter Kollegen als Symptom unserer ...,news,http://www.tageswoche.ch/de/2017_5/leben/74136...,1523,132.177495,84,0.377246,0.340909,0.106252,0.297824,1.122231
27,2017-02-04,741437.0,27.0,Trump treibt Flüchtlinge in eine tödliche Falle,news,http://www.tageswoche.ch/de/2017_5/internation...,941,276.520739,4,0.017964,0.613636,0.222284,0.184014,1.037898
20,2017-02-01,741348.0,17.0,Die 7 wichtigsten Fragen und Antworten zur Spi...,news,http://www.tageswoche.ch/de/2017_5/basel/74134...,2285,183.945422,12,0.053892,0.386364,0.147866,0.446835,1.034956
11,2017-01-31,741271.0,11.0,"Gölä, Trump und Meryl Streep",news,http://www.tageswoche.ch/de/2017_5/schweiz/741...,1986,182.666317,54,0.242515,0.250000,0.146838,0.388365,1.027718
38,2017-02-02,741523.0,24.0,Basel Tattoo wehrt sich kurz vor der Abstimmun...,news,http://www.tageswoche.ch/de/2017_5/basel/74152...,1507,148.556215,12,0.053892,0.545455,0.119418,0.294696,1.013461


In [705]:
top10['week'] = week

In [706]:
top10_series = (top10.groupby(['week'])
       .apply(lambda x: x[['erfolgsindex', 'title', 'url', 'kennzahl_shares', 'kennzahl_comments', 'kennzahl_pageviews', 'kennzahl_readtime']].to_dict('r'))
       .rename(columns={0:'top'}))
top10_series

week
KW 5    [{'kennzahl_shares': 0.3502994011976048, 'kenn...
dtype: object

In [707]:
flop10 = data_subset_erfolgsindex.sort_values('erfolgsindex', ascending=True).head(15)

flop10

,published,article_id,comments,title,type,url,pageviews,readtime,share_count,kennzahl_shares,kennzahl_comments,kennzahl_readtime,kennzahl_pageviews,erfolgsindex
40,2017-02-02,741545.0,0.0,"Wie sehen Sie das, Herr Künzli?",news,http://www.tageswoche.ch/de/2017_5/basel/74154...,9,28.250000,0,0.000000,0.0,0.022709,0.001760,0.024469
34,2017-02-02,741499.0,0.0,Virtuelle Realität – ein mächtiges Spielzeug,news,http://www.tageswoche.ch/de/2017_5/basel/74149...,5,63.750000,0,0.000000,0.0,0.051246,0.000978,0.052224
25,2017-02-01,741420.0,0.0,Jetzt einsteigen: Unser Tippspiel ist wieder a...,news,http://www.tageswoche.ch/de/2017_5/sport/74142...,272,34.277857,7,0.031437,0.0,0.027555,0.053190,0.112182
42,2017-02-04,741610.0,0.0,Der Ball rollt wieder – das Wichtigste zum Sch...,news,http://www.tageswoche.ch/de/2017_5/sport/74161...,245,105.200000,5,0.022455,0.0,0.084566,0.047910,0.154931
33,2017-02-01,741467.0,0.0,DJ Flink erzählt von der DJ-WM in Chile,news,http://www.tageswoche.ch/de/2017_5/kultur/7414...,229,140.373984,0,0.000000,0.0,0.112841,0.044781,0.157622
35,2017-02-02,741502.0,0.0,"Die BaZ macht Politik, BVB streiten mit Mitarb...",news,http://www.tageswoche.ch/de/2017_5/basel/74150...,225,179.069930,0,0.000000,0.0,0.143947,0.043999,0.187946
28,2017-02-02,741450.0,0.0,Feinfühliges Psychogramm einer geheimnisvollen...,news,http://www.tageswoche.ch/de/2017_5/kultur/7414...,594,69.460238,5,0.022455,0.0,0.055836,0.116157,0.194449
16,2017-01-31,741311.0,0.0,"Tramblockade, Chemiehavarien und Mimösli-Gate",news,http://www.tageswoche.ch/de/2017_5/basel/74131...,317,166.726776,0,0.000000,0.0,0.134025,0.061990,0.196014
50,2017-02-05,741697.0,0.0,"Mohamed Elyounoussi: «Ich hoffe, dass diese Po...",news,http://www.tageswoche.ch/de/2017_5/sport/74169...,564,118.935829,0,0.000000,0.0,0.095608,0.110291,0.205898
26,2017-02-01,741426.0,0.0,Nur wenige unsauber deklarierte Nebeneinkünfte...,news,http://www.tageswoche.ch/de/2017_5/basel/74142...,389,150.159296,5,0.022455,0.0,0.120707,0.076069,0.219231


In [708]:
flop10['week'] = week

In [709]:
flop10_series = (flop10.groupby(['week'])
       .apply(lambda x: x[['erfolgsindex', 'title', 'url', 'kennzahl_shares', 'kennzahl_comments', 'kennzahl_pageviews', 'kennzahl_readtime']].to_dict('r'))
       .rename(columns={0:'flop'}))
flop10_series

week
KW 5    [{'kennzahl_shares': 0.0, 'kennzahl_comments':...
dtype: object

In [710]:
cols = "id week nutzer	published_articles	pageviews_news	pageviews_newswire	pageviews_total facebook_interaktionen".split()
j = metaInfo_current[cols].groupby("week").first()
       #.to_json(orient='records', date_format='iso'))
j['top'] = top10_series
j['flop'] = flop10_series
j = j.reset_index()
j = j.to_json(orient='records')

In [711]:
with open('weekly_report/weekly_report.json', 'w') as fp:
    json.dump(json.loads(j), fp, indent=2, sort_keys=True)

print(json.dumps(json.loads(j), indent=2, sort_keys=True))

[
  {
    "facebook_interaktionen": 1325,
    "flop": [
      {
        "erfolgsindex": 0.0244689641,
        "kennzahl_comments": 0.0,
        "kennzahl_pageviews": 0.0017599609,
        "kennzahl_readtime": 0.0227090032,
        "kennzahl_shares": 0.0,
        "title": "Wie sehen Sie das, Herr K\u00fcnzli?",
        "url": "http://www.tageswoche.ch/de/2017_5/basel/741545/wie-sehen-sie-das-herr-kuenzli.htm"
      },
      {
        "erfolgsindex": 0.0522237368,
        "kennzahl_comments": 0.0,
        "kennzahl_pageviews": 0.000977756,
        "kennzahl_readtime": 0.0512459807,
        "kennzahl_shares": 0.0,
        "title": "Virtuelle Realit\u00e4t \u2013 ein m\u00e4chtiges Spielzeug",
        "url": "http://www.tageswoche.ch/de/2017_5/basel/741499/virtuelle-realitaet-ein-maechtiges-spielzeug.htm"
      },
      {
        "erfolgsindex": 0.1121816024,
        "kennzahl_comments": 0.0,
        "kennzahl_pageviews": 0.0531899291,
        "kennzahl_readtime": 0.0275545475,
        "ke